In [1]:
# import numpy as np
import pandas as pd

# from scipy.sparse import csr_matrix
# from sklearn.neighbors import NearestNeighbors

# from pprint import pprint

# from joblib import Parallel, delayed
# import joblib

In [2]:
movie_titles_df = pd.read_csv('title_merge_ratings_500000.csv')

In [3]:
movie_titles_df = movie_titles_df.drop(['Unnamed: 0', 'titleType', 'startYear', 'runtimeMinutes', 'averageRating', 'numVotes', "Action", "Adult", "Adventure", "Animation", "Biography", "Comedy", "Crime", "Documentary", "Drama", "Family", "Fantasy", "Film-Noir", "Game-Show", "History", "Horror", "Music", "Musical", "Mystery", "News", "Reality-TV", "Romance", "Sci-Fi", "Short", "Sport", "Talk-Show", "Thriller", "War", 'Western'], axis=1)

In [4]:
# movie_titles_df['genres'] = movie_titles_df['genres'].str.replace("]", '')
# movie_titles_df['genres'] = movie_titles_df['genres'].str.replace("[", '')
# movie_titles_df['genres'] = movie_titles_df['genres'].str.replace("'", '')
# movie_titles_df['genres'] = movie_titles_df['genres'].str.replace(" ", '')
# movie_titles_df['genres'] = movie_titles_df['genres'].str.replace(",", '|')
# movie_titles_df

In [5]:
# ## Determining how many movies are in each genre

# ## Going through the movies_df dataset and adding each unique genre title to a list
# ## movies with mutliple genres are seperated by a '|', this for loop splits the string at each '|'

# genre_names = set()
# for i in movie_titles_df['genres'].str.split('|').values:
#     genre_names = genre_names.union(set(i))

# ## Function that counts how many times each genre title appears
# def genre_count(dataset, ref_col, count):
#     keyword_count = dict()
#     for i in count: 
#         keyword_count[i] = 0
#     for census_keywords in dataset[ref_col].str.split('|'):   
#         for j in [j for j in census_keywords if j in count]: 
#             keyword_count[j] += 1
                
                
#     ## converting the dict to a list
#     keyword_occurences = []
    
#     ## Adding each count to their respective genre titles and sort the list in decsending order
#     for i,j in keyword_count.items():
#         keyword_occurences.append([i,j])
#     keyword_occurences.sort(key = lambda a:a[1], reverse = True)
#     return keyword_occurences, keyword_count

# ## Calling this function gives access to a list of genre keywords which are sorted by decreasing frequency
# keyword_occurences, dum = genre_count(movie_titles_df, 'genres', genre_names)
# keyword_occurences

In [6]:
## Using TfidfVectorizer to transform text to feature vectors
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfvectorizer = TfidfVectorizer(analyzer='word', stop_words='english')

tfidf_wm = tfidfvectorizer.fit_transform(movie_titles_df['genres'])

In [7]:
## Using Cosine Similarity to calculate similarity between movies based on the genres they have in common
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_wm, tfidf_wm)

## Showing the similarity the first x(5) movies in the dataframe have with each other
cosine_sim[:5, :5]

array([[1.        , 0.15880748, 0.        , 0.        , 0.        ],
       [0.15880748, 1.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.16455827],
       [0.        , 0.        , 0.        , 0.16455827, 1.        ]])

In [41]:
## Building a 1-dimensional array from the movie titles
titles = movie_titles_df['tconst']
indices = pd.Series(movie_titles_df.index, index=movie_titles_df['tconst'])

## Function that retrieves the top recommendations based on the cosine similarity to the movie title entered when fucntion called
def recommend_from(id):
    idx = indices[id]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [42]:
titleID = 'tt0034583'
recommendations = recommend_from(titleID).head(10)
recommendations

25     tt0093058
66     tt0120815
259    tt2713180
291    tt8579674
115    tt0361748
39     tt0109830
56     tt0119217
59     tt0120338
110    tt0332280
207    tt1343092
Name: tconst, dtype: object

In [22]:
dict1 = {'Recommendations': recommendations}

In [28]:
dict1

{'Recommendations': 25     tt0093058
 66     tt0120815
 259    tt2713180
 291    tt8579674
 115    tt0361748
 39     tt0109830
 56     tt0119217
 59     tt0120338
 110    tt0332280
 207    tt1343092
 Name: tconst, dtype: object}

In [45]:
a = recommendations.to_json()

In [46]:
a

'{"25":"tt0093058","66":"tt0120815","259":"tt2713180","291":"tt8579674","115":"tt0361748","39":"tt0109830","56":"tt0119217","59":"tt0120338","110":"tt0332280","207":"tt1343092"}'